In [1]:
import pandas as pd 
import numpy  as np
from wordfreq import word_frequency ,top_n_list,get_frequency_dict

from symspellpy import SymSpell, Verbosity
from itertools import islice

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from random import random, choice
import random
import time

import os
import re
from tqdm import tqdm  
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True,nb_workers=4)


INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
from bnunicodenormalizer import Normalizer 
pandarallel.initialize(progress_bar=True,nb_workers=8)
tqdm.pandas()
bnorm=Normalizer()

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
from typing import Dict, List, Tuple, Any, Union


import json

from datasets import load_metric

from tqdm.auto import tqdm
from IPython.display import display, Audio, HTML
from Levenshtein import distance as lev


cer = load_metric("cer")
wer = load_metric("wer")
import warnings 
warnings.filterwarnings('ignore')

In [27]:
def remove_punctuations(my_str):
    punctuations = '''````£|¢|Ñ+-*/=EROero৳০১২৩৪৫৬৭৮৯012–34567•89।!()-[]{};:'"“\’…,<>.‚/?@#$%^&*_~‘—॥”‰🤣⚽️✌�￰৷￰'''
    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct

def normalize(sen):
    _words = [bnorm(word)['normalized']  for word in sen.split()]
    return " ".join([word for word in _words if word is not None]) 

def wer(ref, hyp ,debug=False):
    r = ref.split()
    h = hyp.split()
    #costs will holds the costs, like in the Levenshtein distance algorithm
    costs = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]
    # backtrace will hold the operations we've done.
    # so we could later backtrace, like the WER algorithm requires us to.
    backtrace = [[0 for inner in range(len(h)+1)] for outer in range(len(r)+1)]

    OP_OK = 0
    OP_SUB = 1
    OP_INS = 2
    OP_DEL = 3

    DEL_PENALTY=1 # Tact
    INS_PENALTY=1 # Tact
    SUB_PENALTY=1 # Tact
    # First column represents the case where we achieve zero
    # hypothesis words by deleting all reference words.
    for i in range(1, len(r)+1):
        costs[i][0] = DEL_PENALTY*i
        backtrace[i][0] = OP_DEL

    # First row represents the case where we achieve the hypothesis
    # by inserting all hypothesis words into a zero-length reference.
    for j in range(1, len(h) + 1):
        costs[0][j] = INS_PENALTY * j
        backtrace[0][j] = OP_INS

    # computation
    for i in range(1, len(r)+1):
        for j in range(1, len(h)+1):
            if r[i-1] == h[j-1]:
                costs[i][j] = costs[i-1][j-1]
                backtrace[i][j] = OP_OK
            else:
                substitutionCost = costs[i-1][j-1] + SUB_PENALTY # penalty is always 1
                insertionCost    = costs[i][j-1] + INS_PENALTY   # penalty is always 1
                deletionCost     = costs[i-1][j] + DEL_PENALTY   # penalty is always 1

                costs[i][j] = min(substitutionCost, insertionCost, deletionCost)
                if costs[i][j] == substitutionCost:
                    backtrace[i][j] = OP_SUB
                elif costs[i][j] == insertionCost:
                    backtrace[i][j] = OP_INS
                else:
                    backtrace[i][j] = OP_DEL

    # back trace though the best route:
    i = len(r)
    j = len(h)
    numSub = 0
    numDel = 0
    numIns = 0
    numCor = 0
    if debug:
        lines = []
        compares = []
    while i > 0 or j > 0:
        if backtrace[i][j] == OP_OK:
            numCor += 1
            i-=1
            j-=1
            if debug:
                lines.append("C\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_SUB:
            numSub +=1
            i-=1
            j-=1
            if debug:
                lines.append("S\t" + r[i]+"\t"+h[j])
        elif backtrace[i][j] == OP_INS:
            numIns += 1
            j-=1
            if debug:
                lines.append("I\t" + "****" + "\t" + h[j])
        elif backtrace[i][j] == OP_DEL:
            numDel += 1
            i-=1
            if debug:
                lines.append("D\t" + r[i]+"\t"+"****")
    return lines

def error_label(ref,hyp):
    li=[]
    lines= wer(ref, hyp,debug=True)
    for line in reversed(lines):
        line = re.sub(r"\t"," ",line)
        line = line.split()
        li.append(line[0])
    return li

def wer_calc(label):
    s=label.count('S')
    d=label.count('D')
    i=label.count('I')
    return int((s+d+i)/len(label)*100)

def masking(arpa):
    sen = arpa.split()
    for words in sen:
        suggestions = sym_spell.lookup(words, Verbosity.CLOSEST,max_edit_distance=0, include_unknown=True)
        for suggestion in suggestions:
            suggestion = str(suggestion)
            suggestion = suggestion.split(',')
            if int(suggestion[2]) == 0:
                mask = '[MASK]'
                sen = [mask if x == words else x for x in sen]
                break
    return " ".join(sen)

def lav_distance(error_word,masked_word):
    return int(lev(error_word, masked_word)/max(len(error_word),len(masked_word))*100)

In [5]:
sym_spell = SymSpell(max_dictionary_edit_distance=5, prefix_length=7)
dictionary_path = 'asr_bangla/data/prothom_alo_word_freq.txt'
sym_spell.load_dictionary(dictionary_path, 0, 1,separator=" ")

True

# Common_voice

In [6]:
df_cv_1 = pd.read_csv('asr_bangla/data/cv_100k_p1.csv')
df_cv_2 = pd.read_csv('asr_bangla/data/cv_100k_p2.csv')
df_cv_3 = pd.read_csv('asr_bangla/data/symspell_nov.csv')
df_cv = pd.read_csv('cv_train.csv')

In [7]:
df_cv_3 = df_cv_3[(df_cv_3.source == 'train') & (df_cv_3.audio == 'noisy')]

In [8]:
df_cv_3 = df_cv_3[['path','wav2vec2','4gram']]

In [9]:
df_cv = df_cv[['path','sentence']]

In [10]:
df_cv_1= df_cv_1.append([df_cv_2,df_cv_3])
df_cv_1 = df_cv_1.reset_index(drop=True)

In [11]:
df_cv_1['path'] = df_cv_1.path.apply(lambda x: os.path.basename(str(x)))

In [12]:
df_cv = df_cv_1.merge(df_cv,on='path',how='inner')

In [13]:
df_cv

,path,wav2vec2,4gram,sentence
0,common_voice_bn_30991371.mp3,দেওয়ান ফরিদ গায়জী বাংলাদেশ আওয়ামী লীগের উপদেষ্...,দেওয়ান ফরিদ গাজী বাংলাদেশ আওয়ামী লীগের উপদেষ্ট...,দেওয়ান ফরিদ গাজী বাংলাদেশ আওয়ামী লীগের উপদেষ...
1,common_voice_bn_30991410.mp3,এর মাধ্যমে কারো গান গাওয়া দক্ষতাবা নিজস্ব ধর্ম...,এর মাধ্যমে কারও গান গাওয়া দক্ষতা বা নিজস্ব ধর্...,এর মাধ্যমে কারও গান গাওয়া দক্ষতা বা নিজস্ব ধর...
2,common_voice_bn_30991513.mp3,ডিজিটাল বিশ্বে তথ্য অন্য যে কোনো ডিজিটাল ফাইলে...,ডিজিটাল বিশ্বে তথ্য অন্য যে কোনো ডিজিটাল ফাইলে...,ডিজিটাল বিশ্বে তথ্য অন্য যে কোনও ডিজিটাল ফাইলে...
3,common_voice_bn_30991535.mp3,হিসাব পাওয়ার পর তিনি স্বল্প সময়ের মধ্যেই সম্পূ...,হিসাব পাওয়ার পর তিনি স্বল্প সময়ের মধ্যেই সম্পূ...,হিসাব পাওয়ার পর তিনি স্বল্প সময়ের মধ্যেই সম্...
4,common_voice_bn_30991592.mp3,নদীর পানিতে বয়ে আসা পলি দলদেশে জমে মাঝে মধ্যে ...,নদীর পানিতে বয়ে আসা পলি তলদেশে জমে মাঝে মধ্যে ...,নদীর পানিতে বয়ে আসা পলি তলদেশে জমে মাঝেমধ্যে ...
...,...,...,...,...
100931,common_voice_bn_31459116.mp3,তিনি বগুড়ার কাঠনাপাড়ায় জন্মগ্রহণ করেন,তিনি বগুড়ার কাঠ না পাড়ায় জন্মগ্রহণ করেন,তিনি বগুড়ার কাটনাপাড়ায় জন্মগ্রহণ করেন।
100932,common_voice_bn_31459130.mp3,তবে ছোমান্টিজ থেকে বেশে থাকতে আমাদের কিছু কর ন...,তবে সমান টিজ থেকে বেশে থাকতে আমাদের কিছু করণীয়...,তবে জন্ডিস থেকে বেঁচে থাকতে আমাদের কিছু করণীয়...
100933,common_voice_bn_31459161.mp3,তিনি অসম উপত্যাকার সািত্য পুরস্কার ও সাহিত্য এ...,তিনি অসম উপত্যাকার সাহিত্য পুরস্কার ও সাহিত্য ...,তিনি অসম উপত্যকা সাহিত্য পুরস্কার ও সাহিত্য অক...
100934,common_voice_bn_31637719.mp3,টি পূর্বে অনেক মুচু ছিল,এটি পূর্বে অনেক উঁচু ছিল,এটি পূর্বে অনেক উঁচু ছিলো।


# Open_SLR

In [14]:
df_slr_1 = pd.read_csv('asr_bangla/data/slr_1_60.csv')
df_slr_2 = pd.read_csv('asr_bangla/data/slr_150_200.csv')
df_slr_3 = pd.read_csv('asr_bangla/data/slr_60_70.csv')
df_slr_4 = pd.read_csv('asr_bangla/data/slr_70_80.csv')
df_slr_5 = pd.read_csv('asr_bangla/data/slr_80_90.csv')
df_slr_6 = pd.read_csv('asr_bangla/data/slr_90_100.csv')

In [15]:
df_slr_1 = df_slr_1.append([df_slr_2,df_slr_3,df_slr_4,df_slr_5,df_slr_6])
df_slr_1 = df_slr_1.reset_index(drop=True)

In [16]:
df_slr_1['path'] = df_slr_1.path.apply(lambda x: os.path.basename(str(x)))

In [17]:
df_slr_1

,path,wav2vec2,4gram
0,000020a912.flac,বাংলাদেশের দায়িত্ব নেবে,বাংলাদেশের দায়িত্ব নেবে
1,000039928e.flac,এই ধরনের কাজ নিয়ে,এই ধরনের কাজ নিয়ে
2,00005debc7.flac,হতে উপার্জিত অর্থ,হতে উপার্জিত অর্থ
3,00009e687c.flac,হাসি বিষল হয়েই আছে,হাসির বিষয় হয়েই আছে
4,00012843bc.flac,শাক দেশ গুরুতে,শাক দেশ গুড়িতে
...,...,...,...
149995,74c6f3b531.flac,একটা বরো কাজ হয়েছে,একটা বড় কাজ হয়েছে
149996,74c702e2d5.flac,জুন্ খালার সোনার দুল,জুন খেলার সোনার দুল
149997,74c706e710.flac,এরপরও বাংলা দেশে,এরপরও বাংলাদেশে
149998,74c70ce063.flac,ছাব্বিশ হাজার নয়শল ছাব্বিশ জনের,ছাব্বিশ হাজার নয় শ ছাব্বিশ জনের


In [18]:
df_slr = pd.read_csv('slr_train.csv')

In [19]:
df_slr['path'] = df_slr.path.apply(lambda x :os.path.basename(str(x)))

In [20]:
df_slr = df_slr_1.merge(df_slr,on='path',how='inner')

In [21]:
df_slr['path'] = 'slr_' + df_slr.path

In [22]:
df_slr

,path,wav2vec2,4gram,sentence
0,slr_000020a912.flac,বাংলাদেশের দায়িত্ব নেবে,বাংলাদেশের দায়িত্ব নেবে,বাংলাদেশে দায়িত্ব নেবে
1,slr_000039928e.flac,এই ধরনের কাজ নিয়ে,এই ধরনের কাজ নিয়ে,এ ধরণের কার্ড নিয়ে
2,slr_00005debc7.flac,হতে উপার্জিত অর্থ,হতে উপার্জিত অর্থ,হতে উপার্জিত অর্থ
3,slr_00009e687c.flac,হাসি বিষল হয়েই আছে,হাসির বিষয় হয়েই আছে,হাসির বিষয় হয়েই আছে
4,slr_00012843bc.flac,শাক দেশ গুরুতে,শাক দেশ গুড়িতে,সার্ক দেশগুলোতে
...,...,...,...,...
149995,slr_74c6f3b531.flac,একটা বরো কাজ হয়েছে,একটা বড় কাজ হয়েছে,একটা বড় কাজ হয়েছে
149996,slr_74c702e2d5.flac,জুন্ খালার সোনার দুল,জুন খেলার সোনার দুল,ঝুনুখালার সোনার দুল
149997,slr_74c706e710.flac,এরপরও বাংলা দেশে,এরপরও বাংলাদেশে,এর পরও বাংলাদেশে
149998,slr_74c70ce063.flac,ছাব্বিশ হাজার নয়শল ছাব্বিশ জনের,ছাব্বিশ হাজার নয় শ ছাব্বিশ জনের,২৬ হাজার ৯শ ২৬ জনের


# Data_Merged

In [23]:
df_actual = df_slr.append(df_cv)

In [24]:
df_actual = df_actual.dropna()

In [25]:
df_actual = df_actual.reset_index(drop=True)

In [26]:
df_actual.rename(columns = {'4gram':'arpa_4gram'}, inplace = True)

In [27]:
df_actual

,path,wav2vec2,arpa_4gram,sentence
0,slr_000020a912.flac,বাংলাদেশের দায়িত্ব নেবে,বাংলাদেশের দায়িত্ব নেবে,বাংলাদেশে দায়িত্ব নেবে
1,slr_000039928e.flac,এই ধরনের কাজ নিয়ে,এই ধরনের কাজ নিয়ে,এ ধরণের কার্ড নিয়ে
2,slr_00005debc7.flac,হতে উপার্জিত অর্থ,হতে উপার্জিত অর্থ,হতে উপার্জিত অর্থ
3,slr_00009e687c.flac,হাসি বিষল হয়েই আছে,হাসির বিষয় হয়েই আছে,হাসির বিষয় হয়েই আছে
4,slr_00012843bc.flac,শাক দেশ গুরুতে,শাক দেশ গুড়িতে,সার্ক দেশগুলোতে
...,...,...,...,...
248612,common_voice_bn_31459116.mp3,তিনি বগুড়ার কাঠনাপাড়ায় জন্মগ্রহণ করেন,তিনি বগুড়ার কাঠ না পাড়ায় জন্মগ্রহণ করেন,তিনি বগুড়ার কাটনাপাড়ায় জন্মগ্রহণ করেন।
248613,common_voice_bn_31459130.mp3,তবে ছোমান্টিজ থেকে বেশে থাকতে আমাদের কিছু কর ন...,তবে সমান টিজ থেকে বেশে থাকতে আমাদের কিছু করণীয়...,তবে জন্ডিস থেকে বেঁচে থাকতে আমাদের কিছু করণীয়...
248614,common_voice_bn_31459161.mp3,তিনি অসম উপত্যাকার সািত্য পুরস্কার ও সাহিত্য এ...,তিনি অসম উপত্যাকার সাহিত্য পুরস্কার ও সাহিত্য ...,তিনি অসম উপত্যকা সাহিত্য পুরস্কার ও সাহিত্য অক...
248615,common_voice_bn_31637719.mp3,টি পূর্বে অনেক মুচু ছিল,এটি পূর্বে অনেক উঁচু ছিল,এটি পূর্বে অনেক উঁচু ছিলো।


In [28]:
df_actual['sentence'] = df_actual['sentence'].parallel_apply(lambda x : remove_punctuations(x))
df_actual["sentence"]=df_actual["sentence"].parallel_apply(lambda x:normalize(x))

In [29]:
#df_actual.to_csv('df_data_actual.csv',index=False)

In [30]:
df_actual['wav2vec2_label'] = df_actual.parallel_apply(lambda x : error_label(x.sentence,x.wav2vec2),axis=1)

In [31]:
df_actual['arpa_label'] = df_actual.parallel_apply(lambda x : error_label(x.sentence,x.arpa_4gram),axis=1)

In [32]:
df_actual['wer_wav2vec2'] = df_actual.wav2vec2_label.parallel_apply(lambda x: wer_calc(x))

In [33]:
df_actual['wer_arpa'] = df_actual.arpa_label.parallel_apply(lambda x: wer_calc(x))

In [34]:
df_actual

,path,wav2vec2,arpa_4gram,sentence,wav2vec2_label,arpa_label,wer_wav2vec2,wer_arpa
0,slr_000020a912.flac,বাংলাদেশের দায়িত্ব নেবে,বাংলাদেশের দায়িত্ব নেবে,বাংলাদেশে দায়িত্ব নেবে,"[S, C, C]","[S, C, C]",33,33
1,slr_000039928e.flac,এই ধরনের কাজ নিয়ে,এই ধরনের কাজ নিয়ে,এ ধরণের কার্ড নিয়ে,"[S, S, S, C]","[S, S, S, C]",75,75
2,slr_00005debc7.flac,হতে উপার্জিত অর্থ,হতে উপার্জিত অর্থ,হতে উপার্জিত অর্থ,"[C, C, C]","[C, C, C]",0,0
3,slr_00009e687c.flac,হাসি বিষল হয়েই আছে,হাসির বিষয় হয়েই আছে,হাসির বিষয় হয়েই আছে,"[S, S, C, C]","[C, C, C, C]",50,0
4,slr_00012843bc.flac,শাক দেশ গুরুতে,শাক দেশ গুড়িতে,সার্ক দেশগুলোতে,"[I, S, S]","[I, S, S]",100,100
...,...,...,...,...,...,...,...,...
248612,common_voice_bn_31459116.mp3,তিনি বগুড়ার কাঠনাপাড়ায় জন্মগ্রহণ করেন,তিনি বগুড়ার কাঠ না পাড়ায় জন্মগ্রহণ করেন,তিনি বগুড়ার কাটনাপাড়ায় জন্মগ্রহণ করেন,"[C, C, S, C, C]","[C, C, I, I, S, C, C]",20,42
248613,common_voice_bn_31459130.mp3,তবে ছোমান্টিজ থেকে বেশে থাকতে আমাদের কিছু কর ন...,তবে সমান টিজ থেকে বেশে থাকতে আমাদের কিছু করণীয়...,তবে জন্ডিস থেকে বেঁচে থাকতে আমাদের কিছু করণীয় আছে,"[C, S, C, S, C, C, C, I, S, S]","[C, I, S, C, S, C, C, C, C, S]",50,40
248614,common_voice_bn_31459161.mp3,তিনি অসম উপত্যাকার সািত্য পুরস্কার ও সাহিত্য এ...,তিনি অসম উপত্যাকার সাহিত্য পুরস্কার ও সাহিত্য ...,তিনি অসম উপত্যকা সাহিত্য পুরস্কার ও সাহিত্য অক...,"[C, C, S, S, C, C, C, S, C, C, S]","[C, C, S, C, C, C, C, S, C, C, S]",36,27
248615,common_voice_bn_31637719.mp3,টি পূর্বে অনেক মুচু ছিল,এটি পূর্বে অনেক উঁচু ছিল,এটি পূর্বে অনেক উঁচু ছিলো,"[S, C, C, S, S]","[C, C, C, C, S]",60,20


In [35]:
df_actual = df_actual[df_actual.wer_wav2vec2 !=0 ]

In [36]:
df_actual.to_csv('asr_bangla/data/asr_cv_slr.csv',index=False)

# Real data masking

In [37]:
df_actual = pd.read_csv('asr_bangla/data/asr_cv_slr.csv')

In [7]:
df_actual = pd.read_csv('asr_bangla/data/asr_cv_slr.csv')

In [8]:
df_actual['masked_arpa'] = df_actual['arpa_4gram'].parallel_apply(lambda x: masking(str(x)))
df_actual['masked_wav2vec2'] = df_actual['wav2vec2'].parallel_apply(lambda x: masking(str(x)))

In [9]:
df_actual

,path,wav2vec2,arpa_4gram,sentence,wav2vec2_label,arpa_label,wer_wav2vec2,wer_arpa,masked_arpa,masked_wav2vec2
0,slr_000020a912.flac,বাংলাদেশের দায়িত্ব নেবে,বাংলাদেশের দায়িত্ব নেবে,বাংলাদেশে দায়িত্ব নেবে,"['S', 'C', 'C']","['S', 'C', 'C']",33,33,বাংলাদেশের দায়িত্ব নেবে,বাংলাদেশের দায়িত্ব নেবে
1,slr_000039928e.flac,এই ধরনের কাজ নিয়ে,এই ধরনের কাজ নিয়ে,এ ধরণের কার্ড নিয়ে,"['S', 'S', 'S', 'C']","['S', 'S', 'S', 'C']",75,75,এই ধরনের কাজ নিয়ে,এই ধরনের কাজ নিয়ে
2,slr_00009e687c.flac,হাসি বিষল হয়েই আছে,হাসির বিষয় হয়েই আছে,হাসির বিষয় হয়েই আছে,"['S', 'S', 'C', 'C']","['C', 'C', 'C', 'C']",50,0,হাসির বিষয় হয়েই আছে,হাসি [MASK] হয়েই আছে
3,slr_00012843bc.flac,শাক দেশ গুরুতে,শাক দেশ গুড়িতে,সার্ক দেশগুলোতে,"['I', 'S', 'S']","['I', 'S', 'S']",100,100,শাক দেশ গুড়িতে,শাক দেশ [MASK]
4,slr_00012880b1.flac,শহজি জাদে,শহরে যারা,সহজেই যাওয়া যায়,"['D', 'S', 'S']","['D', 'S', 'S']",100,100,শহরে যারা,[MASK] [MASK]
...,...,...,...,...,...,...,...,...,...,...
197835,common_voice_bn_31459116.mp3,তিনি বগুড়ার কাঠনাপাড়ায় জন্মগ্রহণ করেন,তিনি বগুড়ার কাঠ না পাড়ায় জন্মগ্রহণ করেন,তিনি বগুড়ার কাটনাপাড়ায় জন্মগ্রহণ করেন,"['C', 'C', 'S', 'C', 'C']","['C', 'C', 'I', 'I', 'S', 'C', 'C']",20,42,তিনি বগুড়ার কাঠ না পাড়ায় জন্মগ্রহণ করেন,তিনি বগুড়ার [MASK] জন্মগ্রহণ করেন
197836,common_voice_bn_31459130.mp3,তবে ছোমান্টিজ থেকে বেশে থাকতে আমাদের কিছু কর ন...,তবে সমান টিজ থেকে বেশে থাকতে আমাদের কিছু করণীয়...,তবে জন্ডিস থেকে বেঁচে থাকতে আমাদের কিছু করণীয় আছে,"['C', 'S', 'C', 'S', 'C', 'C', 'C', 'I', 'S', ...","['C', 'I', 'S', 'C', 'S', 'C', 'C', 'C', 'C', ...",50,40,তবে সমান টিজ থেকে বেশে থাকতে আমাদের কিছু করণীয়...,তবে [MASK] থেকে বেশে থাকতে আমাদের কিছু কর নীয় ...
197837,common_voice_bn_31459161.mp3,তিনি অসম উপত্যাকার সািত্য পুরস্কার ও সাহিত্য এ...,তিনি অসম উপত্যাকার সাহিত্য পুরস্কার ও সাহিত্য ...,তিনি অসম উপত্যকা সাহিত্য পুরস্কার ও সাহিত্য অক...,"['C', 'C', 'S', 'S', 'C', 'C', 'C', 'S', 'C', ...","['C', 'C', 'S', 'C', 'C', 'C', 'C', 'S', 'C', ...",36,27,তিনি অসম উপত্যাকার সাহিত্য পুরস্কার ও সাহিত্য ...,তিনি অসম উপত্যাকার [MASK] পুরস্কার ও সাহিত্য এ...
197838,common_voice_bn_31637719.mp3,টি পূর্বে অনেক মুচু ছিল,এটি পূর্বে অনেক উঁচু ছিল,এটি পূর্বে অনেক উঁচু ছিলো,"['S', 'C', 'C', 'S', 'S']","['C', 'C', 'C', 'C', 'S']",60,20,এটি পূর্বে অনেক উঁচু ছিল,টি পূর্বে অনেক [MASK] ছিল


In [10]:
mask = '[MASK]'

In [11]:
df_actual['arpa_masked_n'] = df_actual['masked_arpa'].parallel_apply(lambda x: str(x).split().count(mask))
df_actual['wav2vec2_masked_n'] = df_actual['masked_wav2vec2'].parallel_apply(lambda x: str(x).split().count(mask))

In [12]:
df_actual['wav2vec2_masked_n'].sum()

299528

In [13]:
df_actual['arpa_masked_n'].sum()

28833

In [14]:
df_actual = df_actual[(df_actual.wav2vec2_masked_n !=0) & (df_actual.wav2vec2_masked_n <=6)]

In [15]:
df_actual = df_actual[['wav2vec2','masked_wav2vec2','wav2vec2_masked_n','sentence']]

In [16]:
df_actual

,wav2vec2,masked_wav2vec2,wav2vec2_masked_n,sentence
2,হাসি বিষল হয়েই আছে,হাসি [MASK] হয়েই আছে,1,হাসির বিষয় হয়েই আছে
3,শাক দেশ গুরুতে,শাক দেশ [MASK],1,সার্ক দেশগুলোতে
4,শহজি জাদে,[MASK] [MASK],2,সহজেই যাওয়া যায়
5,অন্তর্ভুক্ত করেছে রাশিয়াম,অন্তর্ভুক্ত করেছে [MASK],1,অন্তর্ভুক্তও করেছে রাশিয়া
6,যাতে বিভাহ সাদী দিলে,যাতে [MASK] সাদী দিলে,1,যাতে বিবাহ শাদী দিলে
...,...,...,...,...
197835,তিনি বগুড়ার কাঠনাপাড়ায় জন্মগ্রহণ করেন,তিনি বগুড়ার [MASK] জন্মগ্রহণ করেন,1,তিনি বগুড়ার কাটনাপাড়ায় জন্মগ্রহণ করেন
197836,তবে ছোমান্টিজ থেকে বেশে থাকতে আমাদের কিছু কর ন...,তবে [MASK] থেকে বেশে থাকতে আমাদের কিছু কর নীয় ...,1,তবে জন্ডিস থেকে বেঁচে থাকতে আমাদের কিছু করণীয় আছে
197837,তিনি অসম উপত্যাকার সািত্য পুরস্কার ও সাহিত্য এ...,তিনি অসম উপত্যাকার [MASK] পুরস্কার ও সাহিত্য এ...,1,তিনি অসম উপত্যকা সাহিত্য পুরস্কার ও সাহিত্য অক...
197838,টি পূর্বে অনেক মুচু ছিল,টি পূর্বে অনেক [MASK] ছিল,1,এটি পূর্বে অনেক উঁচু ছিলো


In [17]:
df_actual['reference_n'] = df_actual.sentence.parallel_apply(lambda x:len(str(x).split()))

In [18]:
df_actual[df_actual.reference_n == 1]

,wav2vec2,masked_wav2vec2,wav2vec2_masked_n,sentence,reference_n
65,সেপ্টেম্বর তেরো উনিষ চার,সেপ্টেম্বর তেরো [MASK] চার,1,সেপ্টেম্বর,1
104,আধারস পচাতর থেকে অনুসদ চার,[MASK] [MASK] থেকে [MASK] চার,3,থেকে,1
146,গীশ্ব কাল,[MASK] কাল,1,গ্রীষ্মকাল,1
182,ফোরশ,[MASK],1,ফেডারেশন,1
236,খেলা ফত,খেলা [MASK],1,খেলাফত,1
...,...,...,...,...,...
103525,মৌলবাদী জঙ্গীবাদিরা,মৌলবাদী [MASK],1,মৌলবাদীজঙ্গীবাদীরা,1
103529,আটাস একানো থেকে উনিশ্চ বাইস,[MASK] [MASK] থেকে [MASK] [MASK],4,থেকে,1
103558,পরিবা কে ে,[MASK] কে [MASK],2,পরিবারকে,1
166372,গোল টেভিল,গোল [MASK],1,গোলটেবিল,1


In [19]:
df_actual = df_actual[df_actual.reference_n >=2]

In [20]:
wav2vec2_multi_dict = df_actual.set_index('wav2vec2').T.to_dict()
keys = list(wav2vec2_multi_dict.keys())

In [21]:
wav2vec2_multi_dict[keys[0]]

{'masked_wav2vec2': 'হাসি [MASK] হয়েই আছে',
 'wav2vec2_masked_n': 1,
 'sentence': 'হাসির বিষয় হয়েই আছে',
 'reference_n': 4}

In [36]:
def wav2vec2_bert_data(key,masked,reference):
    global df_wav2vec2_temp
    masked_indices = []
    masked = masked.split()
    reference = reference.split()
    for i in range(len(masked)):
        if masked[i] == mask:
            masked_indices.append(i)
    for i in masked_indices:
        sen = key.split()
        error_word = sen[i]
        if i in range(0,len(reference)):
            correct_word = reference[i]
        else:
            correct_word = " "
        sen[i] = mask 
        sen = " ".join(sen)
        temp_dict = {'wav2vec2':key,'masked_wav2vec2':sen,'error_word': error_word,'correct_word':correct_word,'sentence': " ".join(reference) }        
        df_wav2vec2_temp = df_wav2vec2_temp.append(temp_dict,ignore_index=True)

In [40]:
#df_actual.parallel_apply(lambda x: wav2vec2_bert_data(x.wav2vec2,x.masked_wav2vec2,x.sentence), axis=1)

In [44]:
df_wav2vec2_temp_1 = pd.DataFrame()
df_wav2vec2_temp_2 = pd.DataFrame()

In [45]:
for key in tqdm(keys[:75000]):
    masked_indices = []
    masked = wav2vec2_multi_dict[key]['masked_wav2vec2']
    masked = masked.split()
    reference = wav2vec2_multi_dict[key]['sentence']
    reference = reference.split()
    for i in range(len(masked)):
        if masked[i] == mask:
            masked_indices.append(i)
    for i in masked_indices:
        sen = key.split()
        error_word = sen[i]
        if i in range(0,len(reference)):
            correct_word = reference[i]
        else:
            correct_word = " "
        sen[i] = mask 
        sen = " ".join(sen)
        temp_dict = {'wav2vec2':key,'masked_wav2vec2':sen,'error_word': error_word,'correct_word':correct_word,'sentence': " ".join(reference) }
        df_wav2vec2_temp_1 = df_wav2vec2_temp_1.append(temp_dict,ignore_index=True)

  0%|          | 0/75000 [00:00<?, ?it/s]

In [46]:
for key in tqdm(keys[75000:]):
    masked_indices = []
    masked = wav2vec2_multi_dict[key]['masked_wav2vec2']
    masked = masked.split()
    reference = wav2vec2_multi_dict[key]['sentence']
    reference = reference.split()
    for i in range(len(masked)):
        if masked[i] == mask:
            masked_indices.append(i)
    for i in masked_indices:
        sen = key.split()
        error_word = sen[i]
        if i in range(0,len(reference)):
            correct_word = reference[i]
        else:
            correct_word = " "
        sen[i] = mask 
        sen = " ".join(sen)
        temp_dict = {'wav2vec2':key,'masked_wav2vec2':sen,'error_word': error_word,'correct_word':correct_word,'sentence': " ".join(reference) }
        df_wav2vec2_temp_2 = df_wav2vec2_temp_2.append(temp_dict,ignore_index=True)

  0%|          | 0/71603 [00:00<?, ?it/s]

In [48]:
df_wav2vec2_temp = df_wav2vec2_temp_1.append(df_wav2vec2_temp_2,ignore_index=True)

In [49]:
df_wav2vec2_temp

,wav2vec2,masked_wav2vec2,error_word,correct_word,sentence
0,হাসি বিষল হয়েই আছে,হাসি [MASK] হয়েই আছে,বিষল,বিষয়,হাসির বিষয় হয়েই আছে
1,শাক দেশ গুরুতে,শাক দেশ [MASK],গুরুতে,,সার্ক দেশগুলোতে
2,শহজি জাদে,[MASK] জাদে,শহজি,সহজেই,সহজেই যাওয়া যায়
3,শহজি জাদে,শহজি [MASK],জাদে,যাওয়া,সহজেই যাওয়া যায়
4,অন্তর্ভুক্ত করেছে রাশিয়াম,অন্তর্ভুক্ত করেছে [MASK],রাশিয়াম,রাশিয়া,অন্তর্ভুক্তও করেছে রাশিয়া
...,...,...,...,...,...
280686,তিনি বগুড়ার কাঠনাপাড়ায় জন্মগ্রহণ করেন,তিনি বগুড়ার [MASK] জন্মগ্রহণ করেন,কাঠনাপাড়ায়,কাটনাপাড়ায়,তিনি বগুড়ার কাটনাপাড়ায় জন্মগ্রহণ করেন
280687,তবে ছোমান্টিজ থেকে বেশে থাকতে আমাদের কিছু কর ন...,তবে [MASK] থেকে বেশে থাকতে আমাদের কিছু কর নীয় ...,ছোমান্টিজ,জন্ডিস,তবে জন্ডিস থেকে বেঁচে থাকতে আমাদের কিছু করণীয় আছে
280688,তিনি অসম উপত্যাকার সািত্য পুরস্কার ও সাহিত্য এ...,তিনি অসম উপত্যাকার [MASK] পুরস্কার ও সাহিত্য এ...,সািত্য,সাহিত্য,তিনি অসম উপত্যকা সাহিত্য পুরস্কার ও সাহিত্য অক...
280689,টি পূর্বে অনেক মুচু ছিল,টি পূর্বে অনেক [MASK] ছিল,মুচু,উঁচু,এটি পূর্বে অনেক উঁচু ছিলো


In [51]:
df_wav2vec2_temp.to_csv('data/masked.csv',index=False)

In [4]:
df_masked = pd.read_csv('data/masked.csv')

In [5]:
df_raw = pd.read_csv('data/asr_raw.csv')

In [6]:
df_masked

,wav2vec2,masked_wav2vec2,error_word,correct_word,sentence
0,হাসি বিষল হয়েই আছে,হাসি [MASK] হয়েই আছে,বিষল,বিষয়,হাসির বিষয় হয়েই আছে
1,শাক দেশ গুরুতে,শাক দেশ [MASK],গুরুতে,,সার্ক দেশগুলোতে
2,শহজি জাদে,[MASK] জাদে,শহজি,সহজেই,সহজেই যাওয়া যায়
3,শহজি জাদে,শহজি [MASK],জাদে,যাওয়া,সহজেই যাওয়া যায়
4,অন্তর্ভুক্ত করেছে রাশিয়াম,অন্তর্ভুক্ত করেছে [MASK],রাশিয়াম,রাশিয়া,অন্তর্ভুক্তও করেছে রাশিয়া
...,...,...,...,...,...
280686,তিনি বগুড়ার কাঠনাপাড়ায় জন্মগ্রহণ করেন,তিনি বগুড়ার [MASK] জন্মগ্রহণ করেন,কাঠনাপাড়ায়,কাটনাপাড়ায়,তিনি বগুড়ার কাটনাপাড়ায় জন্মগ্রহণ করেন
280687,তবে ছোমান্টিজ থেকে বেশে থাকতে আমাদের কিছু কর ন...,তবে [MASK] থেকে বেশে থাকতে আমাদের কিছু কর নীয় ...,ছোমান্টিজ,জন্ডিস,তবে জন্ডিস থেকে বেঁচে থাকতে আমাদের কিছু করণীয় আছে
280688,তিনি অসম উপত্যাকার সািত্য পুরস্কার ও সাহিত্য এ...,তিনি অসম উপত্যাকার [MASK] পুরস্কার ও সাহিত্য এ...,সািত্য,সাহিত্য,তিনি অসম উপত্যকা সাহিত্য পুরস্কার ও সাহিত্য অক...
280689,টি পূর্বে অনেক মুচু ছিল,টি পূর্বে অনেক [MASK] ছিল,মুচু,উঁচু,এটি পূর্বে অনেক উঁচু ছিলো


In [7]:
df_raw

,path,wav2vec2,arpa_4gram,sentence,wav2vec2_label,arpa_label,wer_wav2vec2,wer_arpa
0,slr_000020a912.flac,বাংলাদেশের দায়িত্ব নেবে,বাংলাদেশের দায়িত্ব নেবে,বাংলাদেশে দায়িত্ব নেবে,"['S', 'C', 'C']","['S', 'C', 'C']",33,33
1,slr_000039928e.flac,এই ধরনের কাজ নিয়ে,এই ধরনের কাজ নিয়ে,এ ধরণের কার্ড নিয়ে,"['S', 'S', 'S', 'C']","['S', 'S', 'S', 'C']",75,75
2,slr_00009e687c.flac,হাসি বিষল হয়েই আছে,হাসির বিষয় হয়েই আছে,হাসির বিষয় হয়েই আছে,"['S', 'S', 'C', 'C']","['C', 'C', 'C', 'C']",50,0
3,slr_00012843bc.flac,শাক দেশ গুরুতে,শাক দেশ গুড়িতে,সার্ক দেশগুলোতে,"['I', 'S', 'S']","['I', 'S', 'S']",100,100
4,slr_00012880b1.flac,শহজি জাদে,শহরে যারা,সহজেই যাওয়া যায়,"['D', 'S', 'S']","['D', 'S', 'S']",100,100
...,...,...,...,...,...,...,...,...
197835,common_voice_bn_31459116.mp3,তিনি বগুড়ার কাঠনাপাড়ায় জন্মগ্রহণ করেন,তিনি বগুড়ার কাঠ না পাড়ায় জন্মগ্রহণ করেন,তিনি বগুড়ার কাটনাপাড়ায় জন্মগ্রহণ করেন,"['C', 'C', 'S', 'C', 'C']","['C', 'C', 'I', 'I', 'S', 'C', 'C']",20,42
197836,common_voice_bn_31459130.mp3,তবে ছোমান্টিজ থেকে বেশে থাকতে আমাদের কিছু কর ন...,তবে সমান টিজ থেকে বেশে থাকতে আমাদের কিছু করণীয়...,তবে জন্ডিস থেকে বেঁচে থাকতে আমাদের কিছু করণীয় আছে,"['C', 'S', 'C', 'S', 'C', 'C', 'C', 'I', 'S', ...","['C', 'I', 'S', 'C', 'S', 'C', 'C', 'C', 'C', ...",50,40
197837,common_voice_bn_31459161.mp3,তিনি অসম উপত্যাকার সািত্য পুরস্কার ও সাহিত্য এ...,তিনি অসম উপত্যাকার সাহিত্য পুরস্কার ও সাহিত্য ...,তিনি অসম উপত্যকা সাহিত্য পুরস্কার ও সাহিত্য অক...,"['C', 'C', 'S', 'S', 'C', 'C', 'C', 'S', 'C', ...","['C', 'C', 'S', 'C', 'C', 'C', 'C', 'S', 'C', ...",36,27
197838,common_voice_bn_31637719.mp3,টি পূর্বে অনেক মুচু ছিল,এটি পূর্বে অনেক উঁচু ছিল,এটি পূর্বে অনেক উঁচু ছিলো,"['S', 'C', 'C', 'S', 'S']","['C', 'C', 'C', 'C', 'S']",60,20


In [8]:
df_prev = pd.read_csv('symspell_nov.csv')

In [10]:
df_prev = df_prev[df_prev.audio == 'noisy']

In [14]:
df_prev

,path,id,source,audio,sentence,duration,snr,wav2vec2,2gram,3gram,...,wav2vec2_label,wav2vec2_wer,3gram_label,3gram_wer,4gram_label,4gram_wer,wav2vec2_sym,2gram_sym,3gram_sym,4gram_sym
0,/kaggle/input/denoiser-for-benchmark/raw/train...,30991599,train,noisy,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,5.544,-44.277168,ছানার সাথে চিনি বা গুর্মীশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,...,"['C', 'C', 'C', 'C', 'D', 'S', 'C', 'C', 'C', ...",15,"['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', ...",0,"['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', ...",0,ছানার সাথে চিনি বা গুর্মীশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...
1,/kaggle/input/denoiser-for-benchmark/raw/train...,30991687,train,noisy,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,4.032,-46.880779,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,...,"['C', 'C', 'C', 'C', 'C', 'C', 'C']",0,"['C', 'C', 'C', 'C', 'C', 'C', 'C']",0,"['C', 'C', 'C', 'C', 'C', 'C', 'C']",0,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক
2,/kaggle/input/denoiser-for-benchmark/raw/train...,30991851,train,noisy,গরীব মানুষকে সে মানুষ বলেই মনে করে না এবং তুচ্...,4.320,-56.581566,গড়ি মানুষ কেসে মানুষ বলি মনে করে না এবং তুচ্ছত...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,...,"['S', 'S', 'S', 'C', 'S', 'C', 'C', 'C', 'C', ...",36,"['I', 'S', 'S', 'C', 'C', 'S', 'C', 'C', 'C', ...",33,"['I', 'S', 'S', 'C', 'C', 'S', 'C', 'C', 'C', ...",33,গড়ি মানুষ কেসে মানুষ বলি মনে করে না এবং তুচ্ছত...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...
3,/kaggle/input/denoiser-for-benchmark/raw/train...,30992090,train,noisy,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,4.284,-29.025093,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,...,"['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C']",0,"['C', 'C', 'C', 'C', 'D', 'C', 'C', 'C']",12,"['C', 'C', 'C', 'C', 'D', 'C', 'C', 'C']",12,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি
4,/kaggle/input/denoiser-for-benchmark/raw/train...,30992661,train,noisy,অদ্যাবধি এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...,5.760,-32.322632,অধ্যাবধী এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,...,"['S', 'C', 'C', 'C', 'C', 'I', 'S', 'C', 'C']",33,"['C', 'C', 'C', 'I', 'S', 'C', 'I', 'S', 'C', ...",40,"['C', 'C', 'C', 'I', 'S', 'C', 'I', 'S', 'C', ...",40,অদ্যাবধি এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38382,/kaggle/input/denoiser-for-benchmark/raw/slr53...,056711c865,openslr,noisy,রাজনৈতিক সমাবেশ,3.100,-37.201962,রাজনৈতিক সমাবেশ,রাজনৈতিক সমাবেশ,রাজনৈতিক সমাবেশ,...,"['C', 'C']",0,"['C', 'C']",0,"['C', 'C']",0,রাজনৈতিক সমাবেশ,রাজনৈতিক সমাবেশ,রাজনৈতিক সমাবেশ,রাজনৈতিক সমাবেশ
38383,/kaggle/input/denoiser-for-benchmark/raw/slr53...,a3230c3684,openslr,noisy,এমন সংসদ সদস্য,3.300,-36.750912,এমন সংসদ সদস্য,এমন সংসদ সদস্য,এমন সংসদ সদস্য,...,"['C', 'C', 'C']",0,"['C', 'C', 'C']",0,"['C', 'C', 'C']",0,এমন সংসদ সদস্য,এমন সংসদ সদস্য,এমন সংসদ সদস্য,এমন সংসদ সদস্য
38384,/kaggle/input/denoiser-for-benchmark/raw/slr53...,7f87c122cb,openslr,no

In [12]:
df_prev[df_prev.source == 'train'] 

,path,id,source,audio,sentence,duration,snr,wav2vec2,2gram,3gram,...,wav2vec2_label,wav2vec2_wer,3gram_label,3gram_wer,4gram_label,4gram_wer,wav2vec2_sym,2gram_sym,3gram_sym,4gram_sym
0,/kaggle/input/denoiser-for-benchmark/raw/train...,30991599,train,noisy,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,5.544,-44.277168,ছানার সাথে চিনি বা গুর্মীশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,...,"['C', 'C', 'C', 'C', 'D', 'S', 'C', 'C', 'C', ...",15,"['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', ...",0,"['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', ...",0,ছানার সাথে চিনি বা গুর্মীশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...
1,/kaggle/input/denoiser-for-benchmark/raw/train...,30991687,train,noisy,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,4.032,-46.880779,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,...,"['C', 'C', 'C', 'C', 'C', 'C', 'C']",0,"['C', 'C', 'C', 'C', 'C', 'C', 'C']",0,"['C', 'C', 'C', 'C', 'C', 'C', 'C']",0,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক
2,/kaggle/input/denoiser-for-benchmark/raw/train...,30991851,train,noisy,গরীব মানুষকে সে মানুষ বলেই মনে করে না এবং তুচ্...,4.320,-56.581566,গড়ি মানুষ কেসে মানুষ বলি মনে করে না এবং তুচ্ছত...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,...,"['S', 'S', 'S', 'C', 'S', 'C', 'C', 'C', 'C', ...",36,"['I', 'S', 'S', 'C', 'C', 'S', 'C', 'C', 'C', ...",33,"['I', 'S', 'S', 'C', 'C', 'S', 'C', 'C', 'C', ...",33,গড়ি মানুষ কেসে মানুষ বলি মনে করে না এবং তুচ্ছত...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...
3,/kaggle/input/denoiser-for-benchmark/raw/train...,30992090,train,noisy,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,4.284,-29.025093,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,...,"['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C']",0,"['C', 'C', 'C', 'C', 'D', 'C', 'C', 'C']",12,"['C', 'C', 'C', 'C', 'D', 'C', 'C', 'C']",12,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি
4,/kaggle/input/denoiser-for-benchmark/raw/train...,30992661,train,noisy,অদ্যাবধি এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...,5.760,-32.322632,অধ্যাবধী এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,...,"['S', 'C', 'C', 'C', 'C', 'I', 'S', 'C', 'C']",33,"['C', 'C', 'C', 'I', 'S', 'C', 'I', 'S', 'C', ...",40,"['C', 'C', 'C', 'I', 'S', 'C', 'I', 'S', 'C', ...",40,অদ্যাবধি এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9931,/kaggle/input/denoiser-for-benchmark/raw/train...,31459116,train,noisy,তিনি বগুড়ার কাটনাপাড়ায় জন্মগ্রহণ করেন,5.580,-24.213869,তিনি বগুড়ার কাঠনাপাড়ায় জন্মগ্রহণ করেন,তিনি বগুড়ার কাঠ না পাড়ায় জন্মগ্রহণ করেন,তিনি বগুড়ার কাঠ না পাড়ায় জন্মগ্রহণ করেন,...,"['C', 'C', 'S', 'C', 'C']",20,"['C', 'C', 'I', 'I', 'S', 'C', 'C']",42,"['C', 'C', 'I', 'I', 'S', 'C', 'C']",42,তিনি বগুড়ার কাটনারপাড়ায় জন্মগ্রহণ করেন,তিনি বগুড়ার কাঠ না পাড়ায় জন্মগ্রহণ করেন,তিনি বগুড়ার কাঠ না পাড়ায় জন্মগ্রহণ করেন,তিনি বগুড়ার কাঠ না পাড়ায় জন্মগ্রহণ করেন
9932,/kaggle/input/denoiser-for-benchmark/raw/train...,31459130,train,noisy,তবে জন্ডিস থেকে বেঁচে থাকতে আমাদে

In [20]:
df_raw[df_raw.sentence.duplicated() == True]

,path,wav2vec2,arpa_4gram,sentence,wav2vec2_label,arpa_label,wer_wav2vec2,wer_arpa
377,slr_009d6e8214.flac,ভোট হবে চব্বিশ ফেব্রুয়ারি,ভোট হবে চব্বিশ ফেব্রুয়ারি,ভোট হবে ফেব্রুয়ারি,"['C', 'C', 'I', 'C']","['C', 'C', 'I', 'C']",25,25
790,slr_0168dba8fc.flac,ার্কিন যুক্তরাস্ট্র চ,মার্কিন যুক্তরাষ্ট্র,মার্কিন যুক্তরাষ্ট্রের চেয়ে,"['S', 'S', 'S']","['C', 'D', 'S']",100,66
806,slr_016c334007.flac,সোলস্য অনন বৈশালে,সোলস অনন বই শালে,সালে,"['I', 'I', 'S']","['I', 'I', 'I', 'S']",100,100
823,slr_01750a3785.flac,কবি ও প্রাবন্ধিক রাজও আলা উদ্দীন,কবি ও প্রাবন্ধিক রাজু আলা উদ্দিন,কবি ও প্রাবন্ধিক রাজু আলাউদ্দিন,"['C', 'C', 'C', 'I', 'S', 'S']","['C', 'C', 'C', 'C', 'I', 'S']",50,33
890,slr_018db2e3b4.flac,দক্ষিণ ভারত মহাসাগরের প্রায় চার্শ বর্গ কিলোমিট...,দক্ষিণ ভারত মহাসাগরের প্রায় চার্ম বর্গ কিলোমিট...,দক্ষিণ ভারত মহাসাগরের প্রায় চারশো বর্গকিলোমিটা...,"['C', 'C', 'C', 'C', 'I', 'S', 'S', 'C', 'C', ...","['C', 'C', 'C', 'C', 'I', 'S', 'S', 'C', 'C', ...",40,33
...,...,...,...,...,...,...,...,...
197829,common_voice_bn_31365386.mp3,কারণ যুক্তরাষ্ট্রের প্রথম কারটি রাজ্য যুক্তরাজ...,কারণ যুক্তরাষ্ট্রের প্রথম কারটি রাজ্য যুক্তরাজ...,কারণ যুক্তরাষ্ট্রের প্রথম তেরটি রাজ্য যুক্তরাজ...,"['C', 'C', 'C', 'S', 'C', 'C', 'C', 'C', 'C']","['C', 'C', 'C', 'S', 'C', 'C', 'C', 'C', 'C']",11,11
197831,common_voice_bn_31378206.mp3,এই জেলাটি আয়তনের দিক থেকে পশ্চিমরঙ্গের বৃহত্তম...,এই জেলাটি আয়তনের দিক থেকে পশ্চিমবঙ্গের বৃহত্তম...,এই জেলাটি আয়তনের দিক থেকে পশ্চিমবঙ্গের বৃহত্তম...,"['C', 'C', 'C', 'C', 'C', 'S', 'C', 'C', 'C', ...","['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', ...",14,0
197832,common_voice_bn_31403271.mp3,এই সামরাজ্যের প্রথান গত প্রথম ৃশ্টিযুদ্ধের মাধ...,এই সাম্রাজ্যের প্রধান গত প্রথম ৃশ্টিযুদ্ধের মা...,এই সাম্রাজ্যের পতন ঘটে প্রথম বিশ্বযুদ্ধের মাধ্...,"['C', 'S', 'S', 'S', 'C', 'S', 'C', 'S', 'S', ...","['C', 'C', 'S', 'S', 'C', 'S', 'C', 'S', 'C', ...",70,29
197834,common_voice_bn_31448771.mp3,জ্যাঙ্গ অভিযোগ অস্বীকার করে,জ্যাঙ্গ অভিযোগ অস্বীকার করে,জ্যাঙ্গো অভিযোগ অস্বীকার করে,"['S', 'C', 'C', 'C']","['S', 'C', 'C', 'C']",25,25


In [27]:
df_sen = df_prev.sentence.tolist()

In [29]:
df_raw = df_raw[~df_raw['sentence'].isin(df_sen)]

In [31]:
df_masked = df_masked[~df_masked['sentence'].isin(df_sen)]

In [33]:
df_masked.to_csv('data/train/masked.csv',index=False)

In [34]:
df_raw.to_csv('data/train/raw.csv',index=False)

In [6]:
df_raw = pd.read_csv('data/train/raw.csv')
df_masked = pd.read_csv('data/train/masked.csv')

In [7]:
df_masked 

,wav2vec2,masked_wav2vec2,error_word,correct_word,sentence
0,হাসি বিষল হয়েই আছে,হাসি [MASK] হয়েই আছে,বিষল,বিষয়,হাসির বিষয় হয়েই আছে
1,শাক দেশ গুরুতে,শাক দেশ [MASK],গুরুতে,,সার্ক দেশগুলোতে
2,শহজি জাদে,[MASK] জাদে,শহজি,সহজেই,সহজেই যাওয়া যায়
3,শহজি জাদে,শহজি [MASK],জাদে,যাওয়া,সহজেই যাওয়া যায়
4,অন্তর্ভুক্ত করেছে রাশিয়াম,অন্তর্ভুক্ত করেছে [MASK],রাশিয়াম,রাশিয়া,অন্তর্ভুক্তও করেছে রাশিয়া
...,...,...,...,...,...
248769,রেডিও বিকিরণের একটি অংশ আপেক্ষিক জেট থেকে আসেয...,রেডিও বিকিরণের একটি অংশ আপেক্ষিক জেট থেকে আসেয...,পদর্শন,পশ্রন,রেডিও বিকিরণের একটি অংশ আপেক্ষিক জেট থেকে আসে ...
248770,আব্দুল মালেক ময়মনসিঙহ জেলার মুক্তিযুদ্ধের সংগঠ...,আব্দুল মালেক [MASK] জেলার মুক্তিযুদ্ধের সংগঠক ...,ময়মনসিঙহ,ময়মনসিংহ,আদ্বুল মালেক ময়মনসিংহ জেলার মুক্তিযুদ্ধের সংগঠ...
248771,তার জয়ের মাধ্যমে প্রথমবারের মতো কোন তৃনমূল কংগ...,তার জয়ের মাধ্যমে প্রথমবারের মতো কোন তৃনমূল কংগ...,পার্থি,পথ্রী,তার জয়ের মাধ্যমে প্রথমবারের মত কোনো তৃণমূল কংগ...
248772,এখানে ল্বযাট ব্যাং্ক পরিষেবা বর্তমান,এখানে [MASK] ব্যাং্ক পরিষেবা বর্তমান,ল্বযাট,ল্বাড,এখানে ল্বাড ব্যাঙ্ক পরিষেবা বর্তমান


# Custom_data_prothom_alo

In [8]:
df_prothom_alo = pd.read_csv('bangla_text_data.csv')

df_prothom_alo= df_prothom_alo.dropna(how='any')

In [9]:
def get_len(sen):
    sen = sen.split()
    l = len(sen)
    return l

In [10]:
#df_prothom_alo = df_prothom_alo.sample(300000)

In [11]:
df_prothom_alo['lena'] = df_prothom_alo['sentence'].apply(lambda x: get_len(str(x)) )

In [12]:
df_prothom_alo.lena.value_counts()

6      790899
5      776612
7      758972
4      708804
8      687480
        ...  
91          1
170         1
217         1
140         1
107         1
Name: lena, Length: 144, dtype: int64

In [13]:
df_prothom_alo = df_prothom_alo[(df_prothom_alo.lena >= 3) & (df_prothom_alo.lena <= 15)]

In [14]:
#df_prothom_alo = df_prothom_alo.sample(1000)

In [15]:
df_prothom_alo.lena.value_counts()

6     790899
5     776612
7     758972
4     708804
8     687480
9     601139
3     592372
10    509464
11    418943
12    341326
13    276272
14    216316
15    172859
Name: lena, dtype: int64

In [16]:
#df_prothom_alo.to_csv('custom_data_for_bert.csv',index=False)

In [17]:
#df_prothom_alo = pd.read_csv('custom_data_for_bert.csv')

In [18]:
df_prothom_alo

,sentence,lena
0,পশ্চিমবঙ্গের বরখাস্ত হওয়া মন্ত্রী পার্থ চট্টোপ...,9
1,আর জামিনের আবেদন করলেন না অর্পিতা,6
3,সেখানে পার্থর আইনজীবী জামিনের আবেদন জানালেও অর...,13
4,অর্পিতা মুখার্জি কারাগারেই,3
7,তাঁর জীবন সংশয়ের আশঙ্কা রয়েছে,5
...,...,...
8418466,হত্যা ও গণহত্যাসহ মানবতাবিরোধী অপরাধের পাঁচটি ...,9
8418467,পাঁচটি অভিযোগই ট্রাইব্যুনালে প্রমাণিত হয়েছে,5
8418468,চারটি অভিযোগে তাঁকে মৃত্যুদণ্ড দেওয়া হয়,6
8418469,এর মধ্যে রাজনৈতিক গ্রুপকে গণহত্যার দায়ের বিষয়ট...,8


In [19]:
character = ['অ','আ','ই','ঈ','উ','ঊ','এ','ঐ','ও','ঔ','ঋ','ঃ','ং','ৎ'
             'ক','খ','গ','ঘ','ঙ','চ', 'ছ','য','জ', 'ঝ','ট','ঠ','ড','র','ড়','ঢ়',
             'ণ','ত','থ','দ','ধ','ন','প','ফ','ব','ভ','ম','স','শ','ষ','য়','হ']

In [20]:
JuktakkhorList = [' ক্ট ' , ' ক্ক ' , ' ক্ত ' , ' ক্য ' , ' ক্র ' , ' ক্ল ' , ' ক্ষ ' , ' ক্ষ্ণ ' , ' ক্ষ্ম ' , ' ক্ষ্য ' , ' ক্স ' , ' খ্র ' , ' গ্ধ ' , ' গ্ধ্য ' , ' গ্ন ' , ' গ্ন্য ' , ' গ্ব ' , ' গ্র ' , ' গ্র্য ' , ' গ্ল ' , ' ঘ্ন ' , ' ঘ্র ' , ' ঙ্ক্য ' , ' ঙ্গ্য ' , ' চ্চ ' , ' চ্ছ্ব ' , ' চ্য ' , ' জ্জ ' , ' জ্জ্ব ' , ' ট্ট ' , ' জ্ব ' , ' জ্য ' , ' জ্র ' , ' ট্য ' , ' ট্র ' , ' ড্ড ' , ' ড্র ' , ' ণ্ট ' , ' ণ্ঠ ' , ' ণ্ড ' , ' ণ্ণ ' , ' ণ্য ' , ' ৎক ' , ' ৎখ ' , ' ত্ত ' , ' ত্ত্ব ' , ' ত্ত্য ' , ' ত্ন ' , ' ৎপ ' , ' ত্ব ' , ' ত্ম ' , ' ত্ম্য ' , ' ত্য ' , ' ত্র ' , ' ত্র্য ' , ' ৎস ' , ' দ্ঘ ' , ' দ্দ ' , ' দ্ধ ' , ' দ্ব ' , ' দ্ভ ' , ' দ্ভ্র ' , ' দ্ম ' , ' দ্য ' , ' দ্র ' , ' দ্র্য ' , ' ধ্ব ' , ' ধ্য ' , ' ধ্র ' , ' ন্ট ' , ' ন্ট্র ' , ' ন্ঠ ' , ' ন্ড ' , ' ন্ড্র ' , ' ন্ত ' , ' ন্ত্ব ' , ' ন্ত্য ' , ' ন্ত্র ' , ' ন্ত্র্য ' , ' ন্থ ' , ' ন্দ ' , ' ন্দ্ব ' , ' ন্দ্র ' , ' ন্ধ ' , ' ন্ন ' , ' ন্য ' , ' প্ট ' , ' প্ত ' , ' প্ন ' , ' প্প ' , ' প্য ' , ' ব্দ ' , ' ব্ধ ' , ' ব্ব ' , ' ব্র ' , ' ভ্য ' , ' ভ্র ' , ' ম্প্র ' , ' ম্ব ' , ' ম্ম ' , ' ম্য ' , ' ম্র ' , ' য্য ' , ' র্ক ' , ' র্গ্য ' , ' র্ঘ্য ' , ' র্জ্য ' , ' র্থ্য ' , ' র্ব্য ' , ' র্খ ' , ' র্গ ' , ' র্ঘ ' , ' র্চ ' , ' র্ছ ' , ' র্জ ' , ' র্ঝ ' , ' র্ট ' , ' র্ড ' , ' র্ণ ' , ' র্ত ' , ' র্থ ' , ' র্দ ' , ' র্দ্ব ' , ' র্দ্র ' , ' র্ধ ' , ' র্ধ্ব ' , ' ল্ট ' , ' ল্ড ' , ' ল্প ' , ' ল্ম ' , ' ল্য ' , ' ল্ল ' , ' শ্ছ ' , ' শ্ন ' , ' শ্ব ' , ' শ্র ' , ' শ্ল ' , ' ষ্ক ' , ' ষ্ক্র ' , ' ষ্ট ' , ' ষ্ট্য ' , ' ষ্ট্র ' , ' ষ্ঠ ' , ' ষ্প ' , ' স্ট ' , ' স্ট্র ' , ' স্ত্র ' , ' স্ত ' , ' স্থ্য ' , ' স্য ' , ' স্র ' , ' স্ল ' , ' হ্র ' , ' হ্ল ' , ' গ্ম ' , ' ম্ভ ' , ' স্ম ' , ' ঙ্ঘ ' , ' ঙ্ক্ষ ' , ' ঙ্খ ' , ' ঙ্ক ' , ' ঙ্‌ক্ত ' , ' ঞ্চ ' , ' ঞ্ছ ' , ' ঞ্জ ' , ' হ্ন ' , ' ঙ্গ ' , ' জ্ঞ ' , ' গ্য ' , ' চ্ছ '] 

In [21]:
ReplaceDict = {
      'ক' : ['ল','য'],
      'খ' : ['কগ','কজ','লহ','ঝ'],
      'গ' : ['ফ','হ'],
      'ঘ' : ['ফগ','হজ'],
      'ঙ' : ['ব','ম'],
      'চ' : ['ভ','চজ','চগ','ভহ'],
      'ছ' : ['ভ','চজ','চগ','ভহ'],
      'জ' : ['ক','হ'],
      'ঝ' : ['কজ','হগ'],
      'ট' : ['র'],
      'ঠ' : ['তজ','তগ','রহ'],
      'ড' : ['স','ফ'],
      'ঢ' : ['দজ','দ্গ','শ','ফহ'],
      'ণ' : ['ব','ম'],
      'ত' : ['র'],
      'থ' : ['তজ', 'তগ', 'রহ'],
      'দ' : ['স', 'ফ'],
      'ধ' : ['দজ', 'দ্গ', 'শ' ,'ফহ'],
      'ন' : ['ব','ম'],
      'প' : ['ও',' ো'],
      'ফ' : ['দ','গ'],
      'ব' : ['ভ','ন'],
      'ভ' : ['‍ব','চ'],
      'ম' : ['ন'],
      'য' : ['হ','ক'],
      'র' : ['এ',' ে','ত'],
      'ল' : ['ক'],
      'শ' : ['সজ','সগ','আহ','ঢ'],
      'ষ' : ['সজ','সগ','আহ','ঢ'],
      'স' : ['আ',' া','দ'],
      'হ' : ['গ','য'],
      'য়' : ['ত','উ','ু'],
      'ড়' : ['এ',' ে','ত'],
      'ঢ়' : ['এ',' ে','ত'],
      'ৎ' : ['র'],
      'ং' : ['ব','ম'],
      'ঃ' : ['গ','য'],
      'অ' : ['প',' ি','ই'],
      'আ' : ['স'],
      'ই' : ['উ','অ'],
      'ঈ' : ['উ','অ'],
      'উ' : ['ই',' ি'],
      'ঊ' : ['ই',' ি'],
      'ঋ' : [''],
      'এ' : ['ও','র'],
      'ঐ' : [''],
      'ও' : ['প','ই'],
      'ঔ' : [''],
      'া' : ['স'],
      'ি' : ['উ','অ'],
      'ো' : ['প',' ি'],
      'ৌ' : [''],
      'ে' : ['ো','র'],
      'ৈ' : ['']
    }

In [22]:
SameClusterDict = {
    'অ' : ['ও','আ','অ্যা','া','ওঁ','ওঃ'],
    'আ' : ['আ','অ্যাঁ','আঃ','ওয়া','এ'],
    'ই':['ই','ঈ','এ','ে','য়'],
    'ঈ':['ই','ঈ','এ','ে','য়'],
    'উ' : ['উ''উঁ','ঊ','ু'],
    'ঊ' : ['উ''উঁ','ঊ','ু'],
    'ঋ' : ['রি','হ্রী','ড়ি','ঢ়ি','ব্রই','ৃ','হ্র'],
    'এ' : ['এ','অ্যা','আঃ'],
    'ঐ': ['অই','ঈ','ওই' , 'ই', 'অ্যাই'],
    'ও': ['ও','আ','অ্যা','া','ওঁ','ওঃ'],
    'ঔ': [ 'ও','আ','অউ','অ্যা','া','ওঁ','ওঃ'],
    'ক': ['ক','খ','গ', 'ও','আ','অ্যা','া','ওঁ','ওঃ','কোঁ',' কো','খ','খো','খও','ক্ষও','কহোঁ','কহও','গোঁ','গয়' ],
    'খ': ['ক','খ','গ', 'ও','আ','অ্যা','া','ওঁ','ওঃ','কোঁ',' কো','খ','খো','খও','ক্ষও','কহোঁ','কহও','গোঁ','গয়' ],
    'গ': ['ক','খ','গ', 'ও','আ','অ্যা','া','ওঁ','ওঃ','কোঁ',' কো','খ','খো','খও','ক্ষও','কহোঁ','কহও','গোঁ','গয়' ],
    'ঘ ': ['খ','খো','খও','ক্ষও','কহোঁ','কহও','গোঁ','গয়' ,'ঘো ','ঘও'],
    'ঙ': ['◌ং','ও','আ','অ্যা','া','ওঁ','ওঃ','অং'],
    'চ': ['চ','ও','আ','অ্যা','া','ওঁ','ওঃ','চোঁ','চও','ছোঁ','ছো','ছও','ছ'],
    'ছ': ['চ','ও','আ','অ্যা','া','ওঁ','ওঃ','চোঁ','চও','ছোঁ','ছো','ছও','ছ'],
    'জ': ['জ','য','জো','যো','ঝ','ঝও'],
    'ঝ': ['জ','য','জো','যো','ঝ','ঝও'],
    'ঞ': ['ন','ঙ','ণ','ঞ','ই','ঈ','এ','ে','য়','নও' ],
    'ট' : ['ত','তঁ','তোঁ','টয়','তো','ট'],
    'ঠ' : ['থ','ত','তঁ','তোঁ','টয়','তো','ট'],
    'ড' : ['দ','দো','দ্য','ডঃ' ,'দ্যো','ঢ','ধ','থ','ত','তঁ','তোঁ','টয়','তো','ট','ঢো','দঃ' ],
    'ঢ': ['দ','দো','দ্য','ডঃ','দ্যো','ঢ','ধ','থ','ত','তঁ','তোঁ','টয়','তো','ট','ঢো','দঃ' ],
    'ণ': ['ন','ঙ','ণ','ঞ','ই','ঈ','এ','ে','য়','নও' ],
    'ত' : ['দ','দো','দ্য','ডঃ' ,'দ্যো','ঢ','ধ','থ','ত','তঁ','তোঁ','টয়','তো','ট','ঢো','দঃ' ],
    'থ': ['দ','দো','দ্য','ডঃ' ,'দ্যো','ঢ','ধ','থ','ত','তঁ','তোঁ','টয়','তো','ট','ঢো','দঃ' ],
    'দ' : ['দ','দো','দ্য','ডঃ' ,'দ্যো','ঢ','ধ','থ','ত','তঁ','তোঁ','টয়','তো','ট','ঢো','দঃ' ],
    'ধ' : ['দ','দো','দ্য','ডঃ' ,'দ্যো','ঢ','ধ','থ','ত','তঁ','তোঁ','টয়','তো','ট','ঢো','দঃ' ],
    'ন': ['ন','ঙ','ণ','ঞ','ই','ঈ','এ','ে','য়','নও' ],
    'প' : ['প','ফ','ব','ভ','ম','ব্য', 'ভঁ', 'বও' ,'বো','বোঁ','ব্যয়','পঅ' ,'পো' ,'পয়' ,'পোঁ','ভঁ' ,'ভঅ' ,'ভয়','ভোঁ','ময়' ,'মোঃ'],
    'ফ' : ['প','ফ','ব','ভ','ম','ব্য', 'ভঁ', 'বও' ,'বো','বোঁ','ব্যয়','পঅ' ,'পো' ,'পয়' ,'পোঁ','ভঁ' ,'ভঅ' ,'ভয়','ভোঁ','ময়' ,'মোঃ'],
    'ব' : ['প','ফ','ব','ভ','ম','ব্য', 'ভঁ', 'বও' ,'বো','বোঁ','ব্যয়','পঅ' ,'পো' ,'পয়' ,'পোঁ','ভঁ' ,'ভঅ' ,'ভয়','ভোঁ','ময়' ,'মোঃ'],
    'ভ' : ['প','ফ','ব','ভ','ম','ব্য', 'ভঁ', 'বও' ,'বো','বোঁ','ব্যয়','পঅ' ,'পো' ,'পয়' ,'পোঁ','ভঁ' ,'ভঅ' ,'ভয়','ভোঁ','ময়' ,'মোঃ'],
    'ম' : ['প','ফ','ব','ভ','ম','ব্য', 'ভঁ', 'বও' ,'বো','বোঁ','ব্যয়','পঅ' ,'পো' ,'পয়' ,'পোঁ','ভঁ' ,'ভঅ' ,'ভয়','ভোঁ','ময়' ,'মোঃ'],
    'য' : ['প','ফ','ব','ভ','ম','ব্য', 'ভঁ', 'বও' ,'বো','বোঁ','ব্যয়','পঅ' ,'পো' ,'পয়' ,'পোঁ','ভঁ' ,'ভঅ' ,'ভয়','ভোঁ','ময়' ,'মোঃ'],
    'র' : ['র','ড়','ঢ়','রও','রোঁ','রো','রয়','ঋ', 'হর'  ],
    'ল' : ['ল','লয়','লো'],
    'শ' : ['শ','ষ','স','ও','আ','অ্যা','া','ওঁ','ওঃ','শো','সঅ','শয়'],
    'ষ' : ['শ','ষ','স','ও','আ','অ্যা','া','ওঁ','ওঃ','শো','সঅ','শয়'],
    'স' : ['শ','ষ','স','ও','আ','অ্যা','া','ওঁ','ওঃ','শো','সঅ','শয়'],
    '‍হ': ['হ','ও','আ','অ্যা','া','ওঁ','ওঃ'],
    'ড়' : ['র','ড়','ঢ়','রও','রোঁ','রো','রয়','ঋ', 'হর'  ],
    '‍ঢ়': ['র','ড়','ঢ়','রও','রোঁ','রো','রয়','ঋ', 'হর'  ],
    '‍য় ': ['ই','ঈ','এ','ে','য়'],

    'ৎ' : ['দ','দো','দ্য','ডঃ' ,'দ্যো','ঢ','ধ','থ','ত','তঁ','তোঁ','টয়','তো','ট','ঢো','দঃ' ],
    'ং' : ['◌ং','ও','আ','অ্যা','া','ওঁ','ওঃ','অং'],
    'ঃ': ['হ','ও','আ','অ্যা','া','ওঁ','ওঃ'],
    '‍ঁ' : [''],

    '‍ি': ['ই','ঈ','এ','ে','য়'],
    'ী' : ['ই','ঈ','এ','ে','য়'],
    'ে':['এ','অ্যা','আঃ'],
    'ৈ'  :['এ','অ্যা','আঃ'],
    'ো' : ['ও','আ','অ্যা','া','ওঁ','ওঃ'],
    'ৌ':['ও','আ','অ্যা','া','ওঁ','ওঃ'],
    'ৃ' : ['রি','হ্রী','ড়ি','ঢ়ি','ব্রই','ৃ','হ্র'],
     '◌ূ'  : ['উ''উঁ','ঊ','ু'],
     ' ু'  : ['উ''উঁ','ঊ','ু']
}

In [28]:
def apply_error(string,error):
    random.seed(time.time()+200)
    errors = ['Cluster','Replace','Juktakkhor','Character','Deletion','Noerror']
    if error == "Cluster":
        if string in list(SameClusterDict.keys()):
            string = random.choice(SameClusterDict[string])
    if error == "Replace":
        if string in list(ReplaceDict.keys()):
            string = random.choice(ReplaceDict[string])
    if error == "Juktakkhor":
        string = random.choice(JuktakkhorList)
    if error == "Character":
        string = random.choice(character)
    if error == "Deletion":
        string = ""
    if error == "insertion":
        temp_error = random.choice(errors)
        string = string + apply_error(string,temp_error)
    if error == "Noerror":
        pass
    return string

In [29]:
def error_generator(word):
    errors = ['Cluster','Replace','Juktakkhor','Character','Deletion','Insertion','Noerror']
    string = []
    string[:0] = word
    random.seed(time.time())
    unlucky_pos = random.randint(0,int(len(word)*0.4))
    noise_index = random.sample(range(len(word)),unlucky_pos)
    for n in range(len(string)):
        if n in noise_index:
            error_t = random.random()
            random.seed(time.time()+100)
            error = random.choice(errors)
            string[n] = apply_error(string[n],error)
    return "".join(string)

In [30]:
sentence = df_prothom_alo.sentence.tolist()

In [31]:
df_custom_error_dataset = pd.DataFrame()
temp_dict = {}
i=0
for sen in tqdm(sentence):
    if len(sen) > 3:
        sen = sen.split()
        random.seed(time.time()+356)
        random_index = random.randint(0,len(sen)-1)
        masked_word = sen[random_index]
        if len(masked_word) > 3:
            error_word = error_generator(masked_word)
    masked_sen = sen[:]
    masked_sen[random_index] = '[MASK]'
    temp_dict[i] = {'sentence': " ".join(sen), "masked_sen": " ".join(masked_sen), "error_word":error_word,"correct_word":masked_word,'similarity' : lav_distance(error_word,masked_word)}
    i=i+1
    #df_custom_error_dataset = df_custom_error_dataset.append(temp_dict,ignore_index=True)

  0%|          | 0/6851458 [00:00<?, ?it/s]

In [32]:
df_custom_error_dataset = df_custom_error_dataset.from_dict(temp_dict,"index")

In [36]:
df_custom_error_dataset

,sentence,masked_sen,error_word,correct_word,similarity
0,পশ্চিমবঙ্গের বরখাস্ত হওয়া মন্ত্রী পার্থ চট্টোপ...,[MASK] বরখাস্ত হওয়া মন্ত্রী পার্থ চট্টোপাধ্যায়...,পশ্চিমবঙ্গের,পশ্চিমবঙ্গের,0
1,আর জামিনের আবেদন করলেন না অর্পিতা,আর জামিনের [MASK] করলেন না অর্পিতা,সবেদন,আবেদন,20
2,সেখানে পার্থর আইনজীবী জামিনের আবেদন জানালেও অর...,সেখানে পার্থর আইনজীবী জামিনের আবেদন জানালেও অর...,সবেদন,না,100
3,অর্পিতা মুখার্জি কারাগারেই,অর্পিতা মুখার্জি [MASK],কারাগ ক্ষ্ম রএই,কারাগারেই,53
4,তাঁর জীবন সংশয়ের আশঙ্কা রয়েছে,তাঁর [MASK] সংশয়ের আশঙ্কা রয়েছে,জী ন্ধ ন,জীবন,62
...,...,...,...,...,...
6851453,হত্যা ও গণহত্যাসহ মানবতাবিরোধী অপরাধের পাঁচটি ...,হত্যা ও গণহত্যাসহ মানবতাবিরোধী অপরাধের পাঁচটি ...,আলোযে,আনা,80
6851454,পাঁচটি অভিযোগই ট্রাইব্যুনালে প্রমাণিত হয়েছে,[MASK] অভিযোগই ট্রাইব্যুনালে প্রমাণিত হয়েছে,পঁচটি,পাঁচটি,16
6851455,চারটি অভিযোগে তাঁকে মৃত্যুদণ্ড দেওয়া হয়,চারটি অভিযোগে তাঁকে [MASK] দেওয়া হয়,মৃত্যুদণ্স,মৃত্যুদণ্ড,10
6851456,এর মধ্যে রাজনৈতিক গ্রুপকে গণহত্যার দায়ের বিষয়ট...,এর মধ্যে রাজনৈতিক গ্রুপকে গণহত্যার দায়ের বিষয়ট...,র য্য েছে,রয়েছে,55


In [37]:
df_custom_error_dataset.rename(columns = {'similarity':'lavenshtein_distance'}, inplace = True)

In [42]:
df_custom_error_dataset = df_custom_error_dataset[(df_custom_error_dataset.lavenshtein_distance > 10 ) & (df_custom_error_dataset.lavenshtein_distance < 90 )]

In [43]:
df_custom_error_dataset.to_csv('data/train/custom_error_dataset.csv',index=False)